In [119]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict, Counter

from sklearn.ensemble import RandomForestRegressor

In [173]:
df_agri = pd.read_csv('/private/tmp/test_agged.csv')
# get uniq lat longs
uniq_latlngs = set(tuple(t) for t in df_agri[['lat', 'lng']].values)
lat_tick = np.median(sorted(np.diff(df_agri['lat'].unique())))
lng_tick = np.median(sorted(np.diff(df_agri['lng'].unique())))
uniq_argi_types = df_agri['type'].unique()

In [124]:
weather_aggs = defaultdict(Counter)
weather_vals = ['prcp', 'tmin', 'tmax', 'vp']
for val in weather_vals:
    df_weather = pd.read_csv('/private/tmp/{}.csv'.format(val), index_col=0)
    for lat, lng in uniq_latlngs:
        lat_mask = (df_weather['lat']<(lat+lat_tick)) & (df_weather['lat']>=(lat))
        lng_mask = (df_weather['lng']>=(lng+lng_tick)) & (df_weather['lng']<(lng))
        weather_aggs[(lat, lng)][val] += df_weather[lat_mask&lng_mask]['val'].mean()

In [146]:
df_weather_by_loc = pd.DataFrame(weather_aggs).T
df_weather_by_loc.index = df_weather_by_loc.index.set_names(['lat', 'lng'])
df_weather_by_loc = df_weather_by_loc.dropna()

In [147]:
df_outs = []
rfr = RandomForestRegressor()
for ag_type in uniq_argi_types:
    df_one_type = df_agri[df_agri['type']==ag_type].set_index(['lat', 'lng'])
    df_train = df_one_type.join(df_weather_by_loc, how='inner')
    rfr.fit(df_train[weather_vals], df_train['ct'])
    ct_ = rfr.predict(df_weather_by_loc[weather_vals])
    df_weather_by_loc['pred__'+str(ag_type)] = ct_
    
    df_out = df_weather_by_loc[['pred__'+str(ag_type)]].reset_index()
    df_out['type']=ag_type
    df_out['ct'] = df_out['pred_corn'].astype(int)
    
    df_out[['lat','lng','type','ct']]

In [170]:
df_out = df_weather_by_loc[['pred_corn']].reset_index()
df_out['type']=1
df_out['ct'] = df_out['pred_corn'].astype(int)

In [171]:
df_out[['lat','lng','type','ct']].to_csv('/private/tmp/test_agged_predictions.csv', index=False)

In [172]:
df_agri

,lat,lng,type,ct
0,37.798731,-123.509596,123,15
1,37.798731,-123.509596,131,407
2,37.798731,-123.509596,69,10
3,37.798731,-123.509596,76,28
4,37.798731,-123.509596,141,3
5,37.798731,-123.509596,142,15
6,37.798731,-123.509596,143,103
7,37.798731,-123.509596,176,3751
8,37.798731,-123.509596,195,46
9,37.798731,-123.509596,152,1242
